<a href="https://colab.research.google.com/github/baalasangar/MLAI/blob/master/13_Capstone/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pydicom
# !pip install -U pip
# !pip install -U setuptools wheel
# !pip install -U "mxnet<2.0.0"
# !pip install autogluon

### Import lib

In [6]:
import pandas as pd 
import numpy as np
import os
import pydicom as dicom
import cv2
import shutil

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import autogluon.core as ag
from autogluon.vision import ObjectDetector

import tensorflow as tf
from tensorflow.keras.models import load_model

### load model

In [ ]:
OBJECT_DETECTION =  '/content/drive/MyDrive/GreatLearning/Captone_project/Code/Baala/model/detector_autog_v1.pkl'
CLASSIFICATION = tf.keras.models.load_model("/content/drive/MyDrive/GreatLearning/Captone_project/Code/Baala/model/resnet152_model.h5")

### load predict data

In [5]:
predict_data = pd.read_csv("/content/drive/MyDrive/GreatLearning/Captone_project/Code/Baala/predict.csv")
predict_data.head()

,image,rois,image_attr
0,/content/drive/MyDrive/GreatLearning/Captone_p...,"[{'class': '0', 'xmin': 0.15625, 'ymin': 0.179...","{'width': 128.0, 'height': 128.0}"
1,/content/drive/MyDrive/GreatLearning/Captone_p...,"[{'class': '0', 'xmin': 0.5546875, 'ymin': 0.3...","{'width': 128.0, 'height': 128.0}"
2,/content/drive/MyDrive/GreatLearning/Captone_p...,"[{'class': '0', 'xmin': 0.1875, 'ymin': 0.2187...","{'width': 128.0, 'height': 128.0}"
3,/content/drive/MyDrive/GreatLearning/Captone_p...,"[{'class': '0', 'xmin': 0.6171875, 'ymin': 0.7...","{'width': 128.0, 'height': 128.0}"
4,/content/drive/MyDrive/GreatLearning/Captone_p...,"[{'class': '0', 'xmin': 0.2265625, 'ymin': 0.2...","{'width': 128.0, 'height': 128.0}"


In [7]:
def CLAHEprocessing(img):
  clahe = cv2.createCLAHE(clipLimit = 5)
  clahe_img = clahe.apply(img) + 30
  return clahe_img

def predict_classification_score(dcm_image_pix):
    X_image_reshaped = [cv2.resize(img,(150,150)) for img in [dcm_image_pix]]
    normalizeimg = lambda x: (np.array(x) / 255.0).astype('float32')
    X_image_reshaped = normalizeimg(X_image_reshaped)
    X_image_reshaped = np.repeat(X_image_reshaped[..., np.newaxis], 3, -1)
    predicted = model.predict(X_image_reshaped)
    return int(round(predicted[0][0]*100,0))    

def predict(img):
      img_enhanced = CLAHEprocessing(img)
      prediction_score = predict_classification_score(img_enhanced)
      infected = "NOT Affected"
      if prediction_score > 50:
        infected = "Affected"  
        fontcolor = "red"
      else:
        prediction_score = 100 - prediction_score
      print("{} ( probability score {} %)".format(infected,prediction_score))   



In [ ]:
for index,row in predict_data.iterrows():
   imgcv = cv2.imread(row["image"])
